### Build a Simple LLM Application with LCEL
This application will translate text from English into another language.

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [2]:
### Open AI API Key and Open Source models--Llama3,Gemma2,mistral--Groq

import os
from dotenv import load_dotenv
load_dotenv()


groq_api_key=os.getenv("GROQ_API_KEY")
print('done')

done


In [4]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021F596B7B10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021F596CA1D0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

from langchain_core.messages import HumanMessage,SystemMessage:
- HumanMessage: Represents a message from the user or human in a conversation. 
- SystemMessage: Represents a message from the system or assistant that provides context, instructions, or guidance to the language model.
- the order in which you declare SystemMessage and HumanMessage (or other message types) matters because it defines the context and flow of the conversation with the language model. Order is dependent on need. 

##### The standard order for declaring messages is:
1.	SystemMessage:
2.	HumanMessage:
3.	AIMessage (Optional): If the conversation has multiple turns, you might include AIMessage to represent the model's previous responses.


In [5]:
from langchain_core.messages import HumanMessage,SystemMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello How are you?")
]

result=model.invoke(messages)

In [6]:
result

AIMessage(content='Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends or family) \n\n\nThe best option depends on the context and your relationship with the person you\'re speaking to.\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 21, 'total_tokens': 109, 'completion_time': 0.16, 'prompt_time': 0.002132195, 'queue_time': 0.016423834000000002, 'total_time': 0.162132195}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-74c46224-0ee8-4bcf-a6f6-56ded478eae0-0', usage_metadata={'input_tokens': 21, 'output_tokens': 88, 'total_tokens': 109})

In [7]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
parser.invoke(result)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, ça va ?** (Informal)\n* **Coucou, comment vas-tu ?** (Very informal, used with friends or family) \n\n\nThe best option depends on the context and your relationship with the person you\'re speaking to.\n'

In [8]:
### Using LCEL- chain the components
chain=model|parser
chain.invoke(messages)

'Here are a few ways to say "Hello, how are you?" in French:\n\n* **Bonjour, comment allez-vous ?** (Formal)\n* **Salut, comment vas-tu ?** (Informal, to someone you know well)\n* **Coucou, ça va ?** (Very informal, to a close friend)\n\n\nLet me know if you\'d like to learn other greetings!\n'

In [9]:
### Prompt Templates
from langchain_core.prompts import ChatPromptTemplate

generic_template="Trnaslate the following into {language}:"

prompt=ChatPromptTemplate.from_messages(
    [("system",generic_template),("user","{text}")]
)

In [10]:
result=prompt.invoke({"language":"French","text":"Hello"})

In [11]:
result.to_messages()

[SystemMessage(content='Trnaslate the following into French:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [12]:
##Chaining together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'